In [9]:
#- 패키지 임포트
import pandas as pd
import numpy as np
#import emoji # 이모지 제거용
import re
from soynlp.normalizer import * #반복되는 자음 제거
import time
from tqdm.notebook import tqdm #for문 진행률 
from time import sleep #for문 진행 시간 
from kiwipiepy import Kiwi #문장 분리2

In [41]:
''' 파일 불러오기 '''
df = pd.read_csv("./산림복지/2018산림복지_본문_노이즈 제거.txt", sep='\t', encoding='UTF-8')
# print(df.shape, type(df)) # 파일 구조 및 컬럼명 확인용
df['org_idx']= df.index # 인덱스 column 만들기: 문장 분리할 때 사용함
# print(df.columns) # org_idx 제대로 추가됐나 단순 확인용.
post_list = list(df['본문']) #자연어 전처리를 위해 본문을 리스트 형태로

In [50]:
''' 불용어 제거 '''
# post_list 양이 많을 경우 불용어 제거 중에 오류 나면 다시 리스트로 만드는 데 오래 걸려서 카피하는 것임. 안 하고 post_list로 진행해도 됨.
post_list2 = post_list.copy()
for i in range(len(post_list2)):
    post_list2[i] = re.sub('\n', ' ', string=str(post_list2[i])) # 줄바꿈을 띄어쓰기 하나로
    post_list2[i] = re.compile('[^|ㄱ-ㅎ|ㅏ-ㅣ|A-Z|a-z|0-9|가-힣]+').sub(' ',post_list2[i])
    post_list2[i] = repeat_normalize(post_list2[i], num_repeats=1)
	# ↑ ㅋㅋㅋ, ㅎㅎㅎ 등의 의미 없는 자음 삭제. num_repeats가 1이면 반복 글자 중 1개만 남기고 2면 2개만 남김.
    post_list2[i] = re.sub(r"^\s+|\s+$", "", post_list2[i]) # 문서 앞뒤 공백 제거
    post_list2[i] = post_list2[i].split() # 문서 내 공백(1개 이상) 기준으로 자르기
    post_list2[i] = ' '.join(post_list2[i]) # 공백 기준으로 나뉜 문서를 다시 1개의 공백을 두고 붙임.
	# ↑ 문서 내 다중 공백을 지우기 위함.]

df['cleaning'] = pd.DataFrame(post_list2) #불용어 제거된 본문을 데이터 프레임으로

In [51]:
df

,url,title,date,본문,org_idx,cleaning
0,https://blog.naver.com/omj0snckzgt6oa/22143129...,영어명언 모음입니다,2018.12.31.,블로그\nmomo 5개의 글\nmomo\n목록열기\n영어명언 모음입니다 momo ...,0,블로그 momo 5개의 글 momo 목록열기 영어명언 모음입니다 momo 2018 ...
1,https://blog.naver.com/bafbef/221431288710,"배리어프리매거진 ,배프베프! 51호 (2018년 12월31일자)",2018.12.31.,",배프베프!\n인사말\nhttps://blog.naver.com/bafbef/221...",1,배프베프 인사말 https blog naver com bafbef 221222451...
2,https://blog.naver.com/woontal8135/221431252254,콩그는 새우은가 마취들되어 일정 기간이 지나나,2018.12.31.,블로그\n게시판 52개의 글\n게시판\n목록열기\n콩그는 새우은가 마취들되어 일정 ...,2,블로그 게시판 52개의 글 게시판 목록열기 콩그는 새우은가 마취들되어 일정 기간이 ...
3,https://blog.naver.com/kwvwbds/221431232747,Home > 주요 계속사업 > 인간상록수 주대 행사 윤일선 박사 전...,2018.12.31.,"블로그\n복지단체,자선가 146개의 글\n복지단체,자선가\n목록닫기\n복지단체,자선...",3,블로그 복지단체 자선가 146개의 글 복지단체 자선가 목록닫기 복지단체 자선가 글 ...
4,https://blog.naver.com/ptkcr/221431018833,평택시 인사(2019년 1월 2일 자),2018.12.31.,◇4급\n▲항만경제전략국장 한병수 ▲도시주택국장 손정호\n◇5급\n▲항만정책과장 정...,4,4급 항만경제전략국장 한병수 도시주택국장 손정호 5급 항만정책과장 정형민 성장전략과...
...,...,...,...,...,...,...
5166,https://blog.naver.com/xo911/221175385072,"정시컨설팅. 가군 서울대와 나군 연세대, 고려대 지원전략과...",2018.01.01.,블로그\n공지 목록\n공지글\n글 제목 작성일\n(1)\n공지 2025학년도 의대(...,5166,블로그 공지 목록 공지글 글 제목 작성일 1 공지 2025학년도 의대 의예과 시행계...
5167,https://blog.naver.com/kohj007/221175370741,“제주환경 살리기..시민들 협조 절실”,2018.01.01.,"포토로그\n포토로그 43,826개의 글\n포토로그\n목록열기\n“제주환경 살리기.....",5167,포토로그 포토로그 43 826개의 글 포토로그 목록열기 제주환경 살리기 시민들 협조...
5168,https://blog.naver.com/mifaffgov/221173727939,오대산 '솔내마을,2018.01.01.,블로그\n공지 목록\n공지글\n글 제목\n작성일\n(481)\n공지 ※독자모집※ 한...,5168,블로그 공지 목록 공지글 글 제목 작성일 481 공지 독자모집 한 주간의 농업 농촌...
5169,https://blog.naver.com/uncnn/221175281491,한국산림복지진흥원 생애주기별 산림복지서비스,2018.01.01.,숲 Issue\n공지 목록\n공지글\n글 제목 작성일\n공지 에코힐링워킹프로그램 숲...,5169,숲 Issue 공지 목록 공지글 글 제목 작성일 공지 에코힐링워킹프로그램 숲해설 숲...


In [52]:
''' 문장 분리 '''
kiwi = Kiwi() #kss보다 속도 빠름 
split_list = [kiwi.split_into_sents(df['cleaning'][i]) for i in tqdm(range(len(df['cleaning'])))]
df['split_list'] = split_list
df

  0%|          | 0/5171 [00:00<?, ?it/s]

,url,title,date,본문,org_idx,cleaning,split_list
0,https://blog.naver.com/omj0snckzgt6oa/22143129...,영어명언 모음입니다,2018.12.31.,블로그\nmomo 5개의 글\nmomo\n목록열기\n영어명언 모음입니다 momo ...,0,블로그 momo 5개의 글 momo 목록열기 영어명언 모음입니다 momo 2018 ...,"[(블로그 momo 5개의 글 momo 목록열기 영어명언 모음입니다, 0, 35, ..."
1,https://blog.naver.com/bafbef/221431288710,"배리어프리매거진 ,배프베프! 51호 (2018년 12월31일자)",2018.12.31.,",배프베프!\n인사말\nhttps://blog.naver.com/bafbef/221...",1,배프베프 인사말 https blog naver com bafbef 221222451...,[(배프베프 인사말 https blog naver com bafbef 2212224...
2,https://blog.naver.com/woontal8135/221431252254,콩그는 새우은가 마취들되어 일정 기간이 지나나,2018.12.31.,블로그\n게시판 52개의 글\n게시판\n목록열기\n콩그는 새우은가 마취들되어 일정 ...,2,블로그 게시판 52개의 글 게시판 목록열기 콩그는 새우은가 마취들되어 일정 기간이 ...,[(블로그 게시판 52개의 글 게시판 목록열기 콩그는 새우은가 마취들되어 일정 기간...
3,https://blog.naver.com/kwvwbds/221431232747,Home > 주요 계속사업 > 인간상록수 주대 행사 윤일선 박사 전...,2018.12.31.,"블로그\n복지단체,자선가 146개의 글\n복지단체,자선가\n목록닫기\n복지단체,자선...",3,블로그 복지단체 자선가 146개의 글 복지단체 자선가 목록닫기 복지단체 자선가 글 ...,[(블로그 복지단체 자선가 146개의 글 복지단체 자선가 목록닫기 복지단체 자선가 ...
4,https://blog.naver.com/ptkcr/221431018833,평택시 인사(2019년 1월 2일 자),2018.12.31.,◇4급\n▲항만경제전략국장 한병수 ▲도시주택국장 손정호\n◇5급\n▲항만정책과장 정...,4,4급 항만경제전략국장 한병수 도시주택국장 손정호 5급 항만정책과장 정형민 성장전략과...,[(4급 항만경제전략국장 한병수 도시주택국장 손정호 5급 항만정책과장 정형민 성장전...
...,...,...,...,...,...,...,...
5166,https://blog.naver.com/xo911/221175385072,"정시컨설팅. 가군 서울대와 나군 연세대, 고려대 지원전략과...",2018.01.01.,블로그\n공지 목록\n공지글\n글 제목 작성일\n(1)\n공지 2025학년도 의대(...,5166,블로그 공지 목록 공지글 글 제목 작성일 1 공지 2025학년도 의대 의예과 시행계...,[(블로그 공지 목록 공지글 글 제목 작성일 1 공지 2025학년도 의대 의예과 시...
5167,https://blog.naver.com/kohj007/221175370741,“제주환경 살리기..시민들 협조 절실”,2018.01.01.,"포토로그\n포토로그 43,826개의 글\n포토로그\n목록열기\n“제주환경 살리기.....",5167,포토로그 포토로그 43 826개의 글 포토로그 목록열기 제주환경 살리기 시민들 협조...,[(포토로그 포토로그 43 826개의 글 포토로그 목록열기 제주환경 살리기 시민들 ...
5168,https://blog.naver.com/mifaffgov/221173727939,오대산 '솔내마을,2018.01.01.,블로그\n공지 목록\n공지글\n글 제목\n작성일\n(481)\n공지 ※독자모집※ 한...,5168,블로그 공지 목록 공지글 글 제목 작성일 481 공지 독자모집 한 주간의 농업 농촌...,[(블로그 공지 목록 공지글 글 제목 작성일 481 공지 독자모집 한 주간의 농업 ...
5169,https://blog.naver.com/uncnn/221175281491,한국산림복지진흥원 생애주기별 산림복지서비스,2018.01.01.,숲 Issue\n공지 목록\n공지글\n글 제목 작성일\n공지 에코힐링워킹프로그램 숲...,5169,숲 Issue 공지 목록 공지글 글 제목 작성일 공지 에코힐링워킹프로그램 숲해설 숲...,[(숲 Issue 공지 목록 공지글 글 제목 작성일 공지 에코힐링워킹프로그램 숲해설...


In [57]:
''' 분리된 문장 한 개를 한 row로 '''
df2 = pd.DataFrame(columns = ['split_str', 'org_idx']) #용량을 줄이기 위해 새로운 데이터프레임으로

''' 문장 데이터 프레임 저장 '''
df2.to_csv("./산림복지/2018산림복지_본문_불용어제거_및_문장분리.csv", index=False, encoding='UTF-8')

In [58]:
df2 = pd.read_csv("./산림복지/2018산림복지_본문_불용어제거_및_문장분리.csv", encoding='UTF-8')
df2

,split_str,org_idx


In [59]:
from IPython.display import clear_output
from IPython import display # 램효율을 늘리기 위해 아웃풋 display를 지워줄 것.
from ipywidgets import Output

out = Output()
display.display(out)

for i in tqdm(range(len(df['split_list']))):
    for j in range(len(df['split_list'][i])):
        split_str= {'split_str': df['split_list'][i][j][0], 'org_idx': [i]}
        split_df= pd.DataFrame(split_str)
        df2 = df2.append(split_df, ignore_index = True)
        df2.to_csv("./산림복지/2018산림복지_본문_불용어제거_및_문장분리.csv", index=False, encoding='UTF-8')
        display.clear_output(wait=True) # 출력지우기

df2.reset_index(drop=True, inplace=True)
df2

C:\Users\user\AppData\Local\Temp\ipykernel_15780\2486653712.py:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append(split_df, ignore_index = True)


In [31]:
df2

In [ ]:
from 맞춤법_검사 import ko_grammar

# 데이터 프레임 나누기
df2_1 = df2[:5000]
df2_2 = df2[5000:10000]
df2_3 = df2[10000:15000]

df2_n = df2[:]

'''인덱스 0부터'''
df2_1.reset_index(drop=True, inplace=True)
df2_2.reset_index(drop=True, inplace=True)
df2_3.reset_index(drop=True, inplace=True)

df2_n.reset_index(drop=True, inplace=True)


'''맞춤법 검사하기''' 
ko_grammar(df2_1['split_str'])
df2_n['correct_str'] = correct_list
df2_n.to_csv('df2_n.txt', encoding='UTF-8')